In [1]:
# Import likely necessary Python libraries for data wrangling and visualisation
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from matplotlib.colors import rgb_to_hsv
from IPython.display import display
# Base path
data_path = "Assets"
# Load blank templates
blank_15_day_df = pd.read_parquet(f"{data_path}/blank_15_day_df.parquet")
blank_15_month_df = pd.read_parquet(f"{data_path}/blank_15_month_df.parquet")
blank_52_day_df = pd.read_parquet(f"{data_path}/blank_52_day_df.parquet")
blank_52_month_df = pd.read_parquet(f"{data_path}/blank_52_month_df.parquet")
blank_60_day_df = pd.read_parquet(f"{data_path}/blank_60_day_df.parquet")
blank_60_month_df = pd.read_parquet(f"{data_path}/blank_60_month_df.parquet")
# Load colormap max values
colormap_max_df = pd.read_parquet(f"{data_path}/colormap_max_df.parquet")
# Load court-specific dataframes
Real_Tennis_Court_df = pd.read_parquet(f"{data_path}/Real_Tennis_Court_df.parquet")
Lawn_Tennis_Court_1_df = pd.read_parquet(f"{data_path}/Lawn_Tennis_Court_1_df.parquet")
Lawn_Tennis_Court_2_df = pd.read_parquet(f"{data_path}/Lawn_Tennis_Court_2_df.parquet")
Rackets_and_Padel_Court_df = pd.read_parquet(f"{data_path}/Rackets_and_Padel_Court_df.parquet")
Sports_Hall_df = pd.read_parquet(f"{data_path}/Sports_Hall_df.parquet")
Squash_C_Court_df = pd.read_parquet(f"{data_path}/Squash_C_Court_df.parquet")
Squash_D_Court_df = pd.read_parquet(f"{data_path}/Squash_D_Court_df.parquet")
Squash_E_Glass_Court_df = pd.read_parquet(f"{data_path}/Squash_E_Glass_Court_df.parquet")
Squash_F_Court_df = pd.read_parquet(f"{data_path}/Squash_F_Court_df.parquet")

# **Seacourt Tennis Club:** Court Time Slot Dashboard

In [2]:
# ---- Run interactive dashboard ----
from ipywidgets import HBox, VBox, interactive_output

# ---- Dropdown widgets ----
cell_totals_dropdown = widgets.Dropdown(
    options=["On", "Off"],
    value="Off",
    description="Cell Totals",
)

height_dropdown = widgets.Dropdown(
    options=list(range(8, 21)),
    value=10,
    description='Graph Height',
)

width_dropdown = widgets.Dropdown(
    options=list(range(8, 21)),
    value=10,
    description='Graph Width',
)

year_dropdown = widgets.Dropdown(
    options=["All", "2022", "2023", "2024", "2025"],  # <- all options now strings
    value="2025",  # <- default as string
    description="Year:"
)

group_by_dropdown = widgets.Dropdown(
    options=["Month", "Day"],
    value="Day",
    description="Group By:"
)

court_dropdown = widgets.Dropdown(
    options=[
        "Real Tennis Court", 
        "Lawn Tennis Court 1", 
        "Lawn Tennis Court 2",
        "Rackets & Padel Court", 
        "Sports Hall",
        "Squash C Court", 
        "Squash D Court", 
        "Squash E (Glass) Court",
        "Squash F Court"
    ],
    value="Lawn Tennis Court 1",
    description="Court:"
)

# ---- Plotting function ----
def plot_court_heatmap(selected_year, selected_groupby, selected_court, graph_height, graph_width, cell_totals):
    selected_year_str = str(selected_year)  # <- Always treat as string

    df_name = selected_court.replace(" ", "_").replace("(", "").replace(")", "").replace("&", "and") + "_df"
    df = globals()[df_name]

    # Filter year (match string version of Year column)
    if selected_year_str != "All":
        df = df[df["Year"].astype(str) == selected_year_str]

    # Pivot table
    heatmap_data = df.groupby(["Time Slot", selected_groupby], observed=True).size().unstack(fill_value=0)

    # Determine blank structure
    if selected_court == "Real Tennis Court":
        blank_key = "blank_60"
    elif selected_court in ["Lawn Tennis Court 1", "Lawn Tennis Court 2"]:
        blank_key = "blank_52"
    else:
        blank_key = "blank_60"

    blank_name = f"{blank_key}_{selected_groupby.lower()}_df"
    blank_df = globals()[blank_name]
    heatmap_data = blank_df.add(heatmap_data, fill_value=0).fillna(0).astype(int)

    # Set y-axis labels dynamically
    n_slots = heatmap_data.shape[0]
    if selected_court == "Real Tennis Court":
        time_labels = [f"{8 + i:02d}:00" for i in range(n_slots)]
    else:
        start_hour = 9 if selected_court in ["Lawn Tennis Court 1", "Lawn Tennis Court 2"] else 8
        slot_minutes = [start_hour * 60 + 15 * i for i in range(n_slots)]
        time_labels = [f"{m // 60:02d}:{m % 60:02d}" for m in slot_minutes]
    heatmap_data.index = time_labels

    # Reorder columns
    if selected_groupby == "Day":
        day_order = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
        heatmap_data = heatmap_data.reindex(columns=day_order)
    elif selected_groupby == "Month":
        month_order = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
                       "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
        existing_months = [m for m in month_order if m in heatmap_data.columns]
        heatmap_data = heatmap_data.reindex(columns=existing_months)

    # ---- Color scale ----
    year_key = selected_year_str if selected_year_str in colormap_max_df["Year"].values else "All"
    vmax_row = colormap_max_df[colormap_max_df["Year"] == year_key].iloc[0]
    vmax_value = vmax_row[selected_groupby]

    # ---- Plot ----
    plt.style.use("dark_background")
    fig, ax = plt.subplots(figsize=(graph_width, graph_height))
    cmap = plt.cm.gist_heat
    sns.set(font_scale=1.0)

    heatmap = sns.heatmap(
        heatmap_data,
        cmap=cmap,
        annot=False,
        fmt="d",
        linewidths=0.5,
        linecolor="#444444",
        cbar_kws={"label": "Booking Count"},
        ax=ax,
        vmin=0,
        vmax=vmax_value
    )

    colorbar = heatmap.collections[0].colorbar
    colorbar.ax.yaxis.label.set_color("white")
    colorbar.ax.tick_params(colors="white")

    if cell_totals == "On":
        vmin, vmax = heatmap.get_children()[0].get_clim()
        for y in range(heatmap_data.shape[0]):
            for x in range(heatmap_data.shape[1]):
                val = heatmap_data.iloc[y, x]
                normed = (val - vmin) / (vmax - vmin) if vmax > vmin else 0
                rgb = cmap(normed)[:3]
                brightness = rgb_to_hsv([[rgb]])[0][0][2]
                text_color = 'black' if brightness > 0.8 else 'white'
                ax.text(x + 0.5, y + 0.5, str(val), ha='center', va='center', color=text_color, fontsize=10)

    ax.set_title(
        f"{selected_court} Usage by {selected_groupby} – {'All Years' if selected_year_str == 'All' else selected_year_str}",
        fontsize=14, color="white"
    )
    ax.set_xlabel(selected_groupby, color="white")
    ax.set_ylabel("Start Time", color="white")
    ax.set_xticklabels(ax.get_xticklabels(), color="white")
    ax.set_yticklabels(ax.get_yticklabels(), color="white", rotation=0)
    plt.tight_layout()
    plt.show()

# ---- Layout ----
row1 = HBox([width_dropdown, height_dropdown, cell_totals_dropdown])
row2 = HBox([group_by_dropdown, court_dropdown, year_dropdown])
controls_ui = VBox([row1, row2])

interactive_plot = interactive_output(
    plot_court_heatmap,
    {
        "selected_year": year_dropdown,
        "selected_groupby": group_by_dropdown,
        "selected_court": court_dropdown,
        "graph_height": height_dropdown,
        "graph_width": width_dropdown,
        "cell_totals": cell_totals_dropdown
    }
)

display(controls_ui, interactive_plot)


Output()